In [1]:
# Check if SparkContext is running.
print sc.version

2.0.1


In [25]:
PATH = "/Users/adarshnair/spark-2.0.1-bin-hadoop2.7/spark_projects/MovieLens"

# Store the u.user data in user_data RDD
user_data = sc.textFile("%s/ml-100k/u.data" % PATH)
print user_data.first()
'''
Data in the form UserID, MovieID, Rating, Timestamp
'''

196	242	3	881250949


'\nData in the form UserID, MovieID, Rating, Timestamp\n'

### Most popular movie in terms of number of times it was rated ###

In [16]:
def parse_lines(line):
    fields = line.split()
    userID, movieID, rating = fields[0], fields[1], fields[2]
    return  (userID, movieID, rating)

In [26]:
'''
Create tuple of the form (movieID, 1) for dataset
'''
movies = user_data.map(lambda x: (int(x.split()[1]), 1))
movies.take(5)

[(242, 1), (302, 1), (377, 1), (51, 1), (346, 1)]

In [27]:
'''
Get counts for each movieID
'''
movie_count = movies.reduceByKey(lambda x, y: x + y)
movie_count.take(100)

[(2, 131),
 (4, 209),
 (6, 26),
 (8, 219),
 (10, 89),
 (12, 267),
 (14, 183),
 (16, 39),
 (18, 10),
 (20, 72),
 (22, 297),
 (24, 174),
 (26, 73),
 (28, 276),
 (30, 37),
 (32, 81),
 (34, 7),
 (36, 13),
 (38, 120),
 (40, 57),
 (42, 148),
 (44, 79),
 (46, 27),
 (48, 117),
 (50, 583),
 (52, 91),
 (54, 104),
 (56, 394),
 (58, 175),
 (60, 64),
 (62, 127),
 (64, 283),
 (66, 162),
 (68, 134),
 (70, 251),
 (72, 129),
 (74, 7),
 (76, 54),
 (78, 33),
 (80, 68),
 (82, 261),
 (84, 18),
 (86, 150),
 (88, 213),
 (90, 95),
 (92, 104),
 (94, 137),
 (96, 295),
 (98, 390),
 (100, 508),
 (102, 54),
 (104, 5),
 (106, 71),
 (108, 65),
 (110, 31),
 (112, 20),
 (114, 67),
 (116, 125),
 (118, 293),
 (120, 67),
 (122, 106),
 (124, 187),
 (126, 97),
 (128, 65),
 (130, 23),
 (132, 246),
 (134, 198),
 (136, 105),
 (138, 19),
 (140, 61),
 (142, 57),
 (144, 243),
 (146, 10),
 (148, 128),
 (150, 157),
 (152, 82),
 (154, 174),
 (156, 148),
 (158, 60),
 (160, 69),
 (162, 106),
 (164, 151),
 (166, 58),
 (168, 316),
 (17

In [24]:
'''
Flip k,v pairs with count as key and movieId as value
'''
flipped = movie_count.map(lambda (x,y): (y,x))
sorted_movie_counts = flipped.sortByKey()
sorted_movie_counts.collect()

[(1, 814),
 (1, 830),
 (1, 852),
 (1, 1122),
 (1, 1130),
 (1, 1156),
 (1, 1236),
 (1, 1310),
 (1, 1320),
 (1, 1340),
 (1, 1348),
 (1, 1352),
 (1, 1364),
 (1, 1366),
 (1, 1414),
 (1, 1452),
 (1, 1458),
 (1, 1460),
 (1, 1476),
 (1, 1482),
 (1, 1486),
 (1, 1492),
 (1, 1494),
 (1, 1498),
 (1, 1510),
 (1, 1520),
 (1, 1526),
 (1, 1536),
 (1, 1546),
 (1, 1548),
 (1, 1562),
 (1, 1564),
 (1, 1566),
 (1, 1568),
 (1, 1570),
 (1, 1572),
 (1, 1574),
 (1, 1576),
 (1, 1580),
 (1, 1582),
 (1, 1584),
 (1, 1586),
 (1, 1596),
 (1, 1604),
 (1, 1606),
 (1, 1614),
 (1, 1616),
 (1, 1618),
 (1, 1624),
 (1, 1626),
 (1, 1630),
 (1, 1632),
 (1, 1634),
 (1, 1636),
 (1, 1638),
 (1, 1640),
 (1, 1648),
 (1, 1650),
 (1, 1654),
 (1, 1660),
 (1, 1666),
 (1, 1668),
 (1, 1670),
 (1, 1674),
 (1, 1676),
 (1, 1678),
 (1, 1680),
 (1, 1682),
 (1, 599),
 (1, 677),
 (1, 711),
 (1, 857),
 (1, 1201),
 (1, 1235),
 (1, 1309),
 (1, 1325),
 (1, 1329),
 (1, 1339),
 (1, 1341),
 (1, 1343),
 (1, 1349),
 (1, 1363),
 (1, 1373),
 (1, 1447),

### Using broadcast variable to get movie names from a different file along with counts###

In [30]:
import pprint
'''
Load movie names
'''
def loadMovieNames():
    movieNames = {}
    with open("/Users/adarshnair/spark-2.0.1-bin-hadoop2.7/spark_projects/MovieLens/ml-100k/u.item") as item_file:
        for line in item_file:
            fields = line.split('|')
            # movieId is key, movie name is value
            movieNames[int(fields[0])] = fields[1]
    return movieNames

In [31]:
'''
Broadcast the dictionary with the movieID: movieName values to all nodes on the cluster
'''
nameDict = sc.broadcast(loadMovieNames())

In [35]:
'''
Get counts of each movie
'''
movies = user_data.map(lambda x: (int(x.split()[1]) , 1))
movie_counts = movies.reduceByKey(lambda x, y: x + y)
flip = movie_counts.map(lambda (x,y): (y,x))
sorted_movie_counts = flip.sortByKey()
result = sorted_movie_counts.collect()

In [40]:
'''
Get movie names
'''
movie_name_counts = sorted_movie_counts.map(lambda (count, movie): (nameDict.value[movie], count))
final_result = movie_name_counts.collect()
pprint.pprint(final_result)

[('Great Day in Harlem, A (1994)', 1),
 ('Power 98 (1995)', 1),
 ('Bloody Child, The (1996)', 1),
 ('They Made Me a Criminal (1939)', 1),
 ("Jupiter's Wife (1994)", 1),
 ('Cyclo (1995)', 1),
 ('Other Voices, Other Rooms (1997)', 1),
 ('Walk in the Sun, A (1945)', 1),
 ('Homage (1995)', 1),
 ('Crude Oasis, The (1995)', 1),
 ('Every Other Weekend (1990)', 1),
 ('Shadow of Angels (Schatten der Engel) (1976)', 1),
 ('Bird of Prey (1996)', 1),
 ('JLG/JLG - autoportrait de d\xe9cembre (1994)', 1),
 ('Coldblooded (1995)', 1),
 ('Lady of Burlesque (1943)', 1),
 ('Damsel in Distress, A (1937)', 1),
 ('Sleepover (1995)', 1),
 ('Raw Deal (1948)', 1),
 ('Gate of Heavenly Peace, The (1995)', 1),
 ('Girl in the Cadillac (1995)', 1),
 ('Window to Paris (1994)', 1),
 ('Mostro, Il (1994)', 1),
 ('Farmer & Chase (1995)', 1),
 ('Mad Dog Time (1996)', 1),
 ('Fear, The (1995)', 1),
 ('Witness (1985)', 1),
 ('Aiqing wansui (1994)', 1),
 ('Shadows (Cienie) (1988)', 1),
 ('The Courtyard (1995)', 1),
 ("Eye of